Формат именования файла домашней работы: FIO_NLP_HW_N.ipynb, где N - номер домашнего задания

## Предобработка текста с помощью Python 

In [2]:
import re

In [3]:
import pandas as pd
import numpy as np

In [ ]:
from nltk import tokenize as tknz
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [4]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [5]:
# Couldn't find what is apostrophe_dict for the 3rd and the 4th task. Used contractions library instead
import contractions

In [6]:
# Couldn't find emoticon_dict.
# Used two dictionaries https://www.kaggle.com/datasets/divyansh22/emoji-dictionary-1 instead and UNICODE codes
# Both those 2 dicts don't have ":)" emoji key
import pickle
with open('../data/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

In [7]:
#Emoji_Dict

In [31]:
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

'#model i love u take with :) u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦'

Осуществим предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания. Для работы объединим train_df и test_df.


In [8]:
train = pd.read_csv('../data/train_tweets.csv')
test = pd.read_csv('../data/test_tweets.csv')

data = pd.concat([train, test], axis=0, ignore_index=True)


In [9]:
data.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [10]:
data.tail()

,id,label,tweet
49154,49155,NaN,thought factory: left-right polarisation! #tru...
49155,49156,NaN,feeling like a mermaid ð #hairflip #neverre...
49156,49157,NaN,#hillary #campaigned today in #ohio((omg)) &am...
49157,49158,NaN,"happy, at work conference: right mindset leads..."
49158,49159,NaN,"my song ""so glad"" free download! #shoegaze ..."


In [11]:
tweets_df = data.drop(['id', 'label'], axis=1)

In [12]:
tweets_df

,tweet
0,@user when a father is dysfunctional and is s...
1,@user @user thanks for #lyft credit i can't us...
2,bihday your majesty
3,#model i love u take with u all the time in ...
4,factsguide: society now #motivation
...,...
49154,thought factory: left-right polarisation! #tru...
49155,feeling like a mermaid ð #hairflip #neverre...
49156,#hillary #campaigned today in #ohio((omg)) &am...
49157,"happy, at work conference: right mindset leads..."


1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". 

In [13]:
def dashrepl(matchobj):
    return ''
re.sub("@[\w]*", dashrepl,
       "@user @user thanks for #lyft credit i can't use cause they don't offer @woah1 vans in pdx. #disapointed #getthanked")

"  thanks for #lyft credit i can't use cause they don't offer  vans in pdx. #disapointed #getthanked"

In [14]:
re.sub("@[\w]*", "",
       "@user @user thanks for #lyft credit i can't use cause they don't offer @woah1 vans in pdx. #disapointed #getthanked")

"  thanks for #lyft credit i can't use cause they don't offer  vans in pdx. #disapointed #getthanked"

In [36]:
re.sub(r'[^a-zA-Z0-9]', " ",
      "@user @user thanks for #lyft c4684redit i can't 4864 @woah1 vans in pdx. #disapointed #getthanked")

' user  user thanks for  lyft c4684redit i can t 4864 use cause they don t offer  woah1 vans in pdx   disapointed  getthanked'

In [37]:
re.sub(r'[^a-zA-Z]', " ",
      "@user @user thanks for #lyft c4684redit i can't 4864 @woah1 vans in pdx. #disapointed #getthanked")

' user  user thanks for  lyft c    redit i can t      use cause they don t offer  woah  vans in pdx   disapointed  getthanked'

In [39]:
def preprocess_text(texts_list): # texts_list - an iterable of tweets
    prepared_texts = []
    
    for text in texts_list:
        
        # 1) Удалим @user из всех твитов с помощью паттерна "@[\w]*". 
        text = re.sub("@[\w]*", "", text)
        
        # 2) Изменим регистр твитов на нижний с помощью .lower()
        text = text.lower()
        
        # 3) Заменим сокращения с апострофами (пример: ain't, can't) на пробел & 4)Заменим сокращения на их полные формы
        expanded_words = []   
        for word in text.split():
            # using contractions.fix to expand the shortened words
            expanded_words.append(contractions.fix(word))            
        text = ' '.join(expanded_words)
        
        # 5) Заменим эмотиконы (пример: ":)" = "happy") на пробелы 
        # 
        for emot in Emoji_Dict:
            text = re.sub(r'('+emot+')', " " , text)
        
        # doing the same with UNICODE emoji_pattern
        text = emoji_pattern.sub(r' ', text)
            
        # 6) Заменим пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'
        text = re.sub(r'[^\w\s]', "", text)
        
        # 7) Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'.
        # 8) Заменим числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'
        # the result of 8 overlaps 7 (and 6 by the way), so we'll do 8th
        text = re.sub(r'[^a-zA-Z]', " ", text)
        
        # 9) Удалим из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])
        
        text = ' '.join([w for w in text.split() if len(w)>1])   
        
    prepared_texts.append(text)
    return prepared_texts

Этот метод ищет по заданному шаблону в начале строки. Например, если мы вызовем метод match() на строке «find matched pattern» с шаблоном «find», то он завершится успешно. Однако если мы будем искать «matched», то результат будет отрицательный.

In [ ]:
 tweets_df['prepared_tweet'] = preprocess_text(tweets_df['tweet'])

In [ ]:
 tweets_df

10) Поделим твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [ ]:
tweets_df['tweet_token'] = [nltk.tokenize.word_tokenize(tweet) for tweet in tweets_df['tweet']]    

11) Удалим стоп-слова из токенов, используя nltk.corpus.stopwords. Создадим столбец 'tweet_token_filtered' без стоп-слов.

In [ ]:
tweets_df['tweet_token_filtered'] = [nltk.corpus.stopwords(tweet) for tweet in tweets_df['tweet_token']]

In [ ]:
12) Применим стемминг к токенам с помощью nltk.stem.PorterStemmer. Создадим столбец 'tweet_stemmed' после применения стемминга.

In [ ]:

13) Применим лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

14) Сохраним результат предобработки в pickle-файл.